<a href="https://colab.research.google.com/github/KLPorter10/Global-Suicide-Mental-Health-Substance-Abuse---Junior-Seminar-Data-Science-Project/blob/main/Junior_Seminar_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd

# load tax dataset
tax_data_path = 'Tennessee Tax Income Zipcode.csv'
tax_data = pd.read_csv(tax_data_path)

#Load police reports dara
police_reports_path = '/content/Metro_Nashville_Police_Department_Incidents_20240426.csv'
police_reports = pd.read_csv(police_reports_path)

#Display the first few rows of each dataset to verify the import
print("Tax Data:")
print(tax_data.head())

print("\nPolice Reports Data:")
print(police_reports.head())



Tax Data:
  ZIP Code Number Returns   Amount
0    37010            590  235,978
1    37010            140    10786
2    37010            110    17989
3    37010            100    21309
4    37010             80    27613

Police Reports Data:
   Latitude  Longitude  ZIP Code      Offense Description Victim Ethnicity  \
0    36.050    -86.650       NaN            VEHICLE THEFT     Non-Hispanic   
1    36.165    -86.769   37213.0           FOUND PROPERTY          Unknown   
2    36.160    -86.794   37203.0            VEHICLE THEFT         Hispanic   
3    36.170    -86.780       NaN  BURGLARY- MOTOR VEHICLE     Non-Hispanic   
4    36.170    -86.780       NaN  BURGLARY- MOTOR VEHICLE     Non-Hispanic   

          Mapped Location  
0    POINT (-86.65 36.05)  
1  POINT (-86.769 36.165)  
2   POINT (-86.794 36.16)  
3    POINT (-86.78 36.17)  
4    POINT (-86.78 36.17)  


In [37]:
# Get an overview of the tax data
print("Tax Data Overview:")
print(tax_data.info())


# Get summary statistics for numerical colums
print("\nTax Data Summary Statistics:")
print(tax_data.describe())

#Check for missing values
print("\nMissing Values in Tax Data:")
print(tax_data.isnull().sum())

#Get an overview of the police reports data
print("\n\nPolice Reports Dara Overview:")
print(police_reports.info())

#Get summary statistics for numerical columns
print("\nPolice Reports Dara Summary Statistics:")
print(police_reports.describe())

#Check for missing values
print("\nMissing Values in Police Reports Data:")
print(police_reports.isnull().sum())


Tax Data Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4757 entries, 0 to 4756
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ZIP Code        4167 non-null   object
 1   Number Returns  4137 non-null   object
 2   Amount          4137 non-null   object
dtypes: object(3)
memory usage: 111.6+ KB
None

Tax Data Summary Statistics:
       ZIP Code Number Returns Amount
count      4167           4137   4137
unique      619            343   3819
top       37010             **     **
freq          7            564    227

Missing Values in Tax Data:
ZIP Code          590
Number Returns    620
Amount            620
dtype: int64


Police Reports Dara Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888434 entries, 0 to 888433
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Latitude             870994 non

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


# Handling missing values in police reports data
# If ZIP code is missing, use reverse geocoding to impute it based on latitude and longitude
geolocator = Nominatim(user_agent="police_reports")

def get_zipcode(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        address = location.raw['address']
        return address.get('postcode')
    except (GeocoderTimedOut, ValueError):
        return None

def fill_missing_zip(row):
    if pd.isnull(row['ZIP Code']):
        return get_zipcode(row['Latitude'], row['Longitude'])
    else:
        return row['ZIP Code']

police_reports['ZIP Code'] = police_reports.apply(fill_missing_zip, axis=1)

# Cleaning data - remove rows with missing ZIP codes
police_reports.dropna(subset=['ZIP Code'], inplace=True)

# Display first few rows to verify changes
print("Cleaned Police Reports Data:")
print(police_reports.head())

# on to feature engineering


In [50]:
# Feature Engineering for Tax Data
# Convert 'Amount' column to numeric
tax_data['Amount'] = tax_data['Amount'].str.replace(',', '').astype(float)

# Select relevant features
tax_features = ['ZIP Code', 'Number Returns', 'Amount']

# Subset the tax data with selected features
tax_model_data = tax_data[tax_features]

# Display the selected features for tax data
print("Tax Model Data:")
print(tax_model_data.head())

# Feature Engineering for Police Reports Data
# Convert 'Amount' column to numeric
police_reports['Amount'] = police_reports['Amount'].str.replace(',', '').astype(float)

# Select relevant features
police_features = ['Latitude', 'Longitude', 'Offense Description', 'Victim Ethnicity']

# Subset  poolice reports data with selected features
police_model_data = police_reports[police_features]

# Display  selected features for police reports data
print("\nPolice Reports Model Data:")
print(police_model_data.head())


ValueError: could not convert string to float: '**'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Merge tax data and police reports data based on ZIP code
#figure out why data wont merge save for later

# Split the data into training and testing sets
X = model_data.drop('Amount', axis=1)
y = model_data['Amount']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
rf_y_pred = rf_model.predict(X_test)

# Evaluate the Random Forest model
rf_mse = mean_squared_error(y_test, rf_y_pred)
rf_r2 = r2_score(y_test, rf_y_pred)

print("Random Forest Mean Squared Error:", rf_mse)
print("Random Forest R-squared:", rf_r2)
